In [1]:
from ranker import *

/Users/svjack/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.ensemble.bagging module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.ensemble. Anything that cannot be imported from sklearn.ensemble is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/svjack/opt/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neural_network.multilayer_perceptron module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neural_network. Anything that cannot be imported from sklearn.neural_network is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/svjack/opt/anaconda3/lib/python3.8/site-packages/sklearn/base.py:313: UserWarning: Trying to unpickle estimator M

In [2]:
sim_model = SentenceTransformer('LaBSE')
sim_model.pool = None

br_cls = RFR(b_clf,
        all_cate_list=list(pid_zh_b_dict.values()),
       sim_model=sim_model
       )

Batches:   0%|          | 0/25 [00:00<?, ?it/s]

In [134]:
property_info_df = load_pickle("property_info_df.pkl")

In [135]:
property_info_df = property_info_df[
    property_info_df["pid"].isin(list(pid_zh_b_dict.keys()))
]

In [136]:
property_info_df["zh_info_str"] = property_info_df["pid"].map(
    lambda x: pid_zh_b_dict[x]
)

In [137]:
property_info_df["en_info"] = property_info_df["en_info"].map(
    lambda x: [sorted(x, key = len)[0]] if x else []
)

In [138]:
property_info_df["zh_info"] = property_info_df["zh_info_str"].map(
    lambda x: list(map(lambda y: y.strip(), x.split(" ")))
)

In [139]:
property_info_df = property_info_df[
    property_info_df.apply(lambda x: x["en_info"] and x["zh_info"], axis = 1).map(bool)
]

In [142]:
####property_info_df

In [95]:
br_cls.produce_rank_df("美国的货币是什么？",  br_cls.all_cate_list).head(2)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


,prob,cate
762,0.991404,货币 貨幣 項目使用的貨幣 项目使用的货币
58,0.971753,色彩空间 颜色空间


In [100]:
pd.merge(
    br_cls.produce_rank_df("美国的货币是什么？",  br_cls.all_cate_list).head(2), 
    property_info_df,
    left_on = "cate", right_on = "zh_info"
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


,prob,cate,pid,en_info,zh_info
0,0.991404,货币 貨幣 項目使用的貨幣 项目使用的货币,P38,currency currency used by item base currency u...,货币 貨幣 項目使用的貨幣 项目使用的货币
1,0.971753,色彩空间 颜色空间,P929,color space colour space mathematical model de...,色彩空间 颜色空间


In [ ]:
property_info_df_str = property_info_df.applymap(
    lambda x: if type(x) == type()
)

In [7]:
from simplet5 import SimpleT5

model = SimpleT5()
model.from_pretrained(model_type="mt5", model_name="google/mt5-base")

model.load_model(
    "mt5",
    "/Users/svjack/temp/kb_aug/model/t5_gen_outputs/nq-simplet5-epoch-2-train-loss-0.0072-val-loss-0.0045",
    use_gpu = False
)

In [9]:
import sqlite_utils

In [10]:
assert os.path.exists("kbqa-explore/multi_lang_kb_dict.db")
wiki_entity_db = sqlite_utils.Database("kbqa-explore/multi_lang_kb_dict.db")
assert "en_zh_so_search" in wiki_entity_db.table_names()

In [11]:
sent = "碳酸钙的副产品是什么? 在温度为1000的时候?"

In [101]:
from fuzzywuzzy import fuzz

/Users/svjack/opt/anaconda3/lib/python3.8/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [35]:
import jieba.posseg as posseg
def unzip_string(x, size = 2):
    if len(x) <= size:
        return [x]
    req = []
    for i in range(len(x) - size + 1):
        req.append(x[i: i + size])
    return req

In [86]:
def iter_sent_on_db(sent, db = wiki_entity_db, maintain_flag = ["n", "v"], refine_token_size = 2,
                   rm_keys = ["是"], take_one = True, rp_eng = True
                   ):
    sent_c = set(map(lambda tt2: tt2[0] ,filter(lambda t2: 
                    any(map(lambda f: t2[1].startswith(f), maintain_flag))
                    ,map(lambda x: (x.word, x.flag) ,posseg.lcut(sent)))))
    
    sent_c = pd.Series(list(sent_c)).map(
        lambda x: unzip_string(x, refine_token_size)
    ).explode().dropna().drop_duplicates().tolist()
    
    if not sent_c:
        return []
    l = list(map(
        lambda x: (x, 
        list(map(lambda y: y["o"] ,
        wiki_entity_db.query("select * from en_zh_so_search where o like '%{}%'".format(x)))))
        , sent_c))
    #return l
    l = list(map(lambda t2: (
        t2[0], 
        sorted(set(filter(lambda x: x in sent, t2[1])), key = len, reverse = True)
    ), l))
    ll = []
    for k, v in l:
        if v and k not in rm_keys:
            vvv = []
            #print(v)
            for vv in v:
                assert type(vv) == type("")
                vl = list(wiki_entity_db.query("select s from en_zh_so_search where o = '{}'".format(vv)))
                sl = list(map(lambda x: x["s"], vl))
                if take_one:
                    sl = [sorted(sl, key = lambda x:int(x[1:]))[0]]
                    if rp_eng:
                        sl = list(wiki_entity_db.query("select o from en_zh_so_search where s = '{}' and lang = 'en'".format(sl[0])))
                        sl = list(map(lambda x: x["o"], sl))
                vvv.extend(sl)
                #print(vvv)
            ll.append((k, vvv))
    return dict(ll)

In [147]:
def entity_property_simplify_extraction(
    sent, br_cls, property_topk = 3
):
    print("fit entity")
    entity_part = iter_sent_on_db(sent)
    entity_part = dict(
        map(lambda t2: (t2[0], [sorted(t2[1], key = len)[0]] if t2[1] else []) ,entity_part.items())
    )
    print("pred prop")
    
    property_part = br_cls.produce_rank_df(sent,  br_cls.all_cate_list).head(property_topk)
    #print("0 :")
    #print(property_part)
    property_part = pd.merge(
    property_part, 
    property_info_df,
    left_on = "cate", right_on = "zh_info_str"
)[["pid", "zh_info", "en_info", "prob"]]
    #print("1 :")
    #print(property_part)
    property_part["en_info"] = property_part["en_info"].map(
        lambda x: [sorted(x, key = len)[0]]
    )
    property_part["zh_info"] = property_part["zh_info"].map(
        lambda x: [sorted(x, key = lambda y: fuzz.ratio(y, sent), reverse = True)[0]]
    )
    return entity_part, property_part

In [148]:
a, b = entity_property_simplify_extraction("宁波在哪个省？", br_cls)

fit entity
pred prop


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s finished


In [149]:
a

{'宁波': ['Ningbo'], '省': ['département']}

In [150]:
b

,pid,zh_info,en_info,prob
0,P131,[所在省],[in],0.991715
1,P1001,[辖区],[jurisdiction],0.989723
2,P150,[下辖],[contains],0.972107


In [151]:
"宁波在哪个省" + " * " + "{'Ningbo': '宁波'}" + "|" + "{'in': '所在省'}"

"宁波在哪个省 * {'Ningbo': '宁波'}|{'in': '所在省'}"

In [162]:
model.predict(
    "宁波在哪个省" + " * " + "{'Ningbo': '宁波'}" + "|" + "{'in': '所在省'}"
)

['select distinct?answer where {?answer wdt:in wd:Ningbo}']

In [163]:
model.predict(
    "宁波在哪个省" + " * " + "{'宁波': '宁波'}" + "|" + "{'所在省': '所在省'}"
)

['select distinct?answer where {?answer wdt:所在_省 wd:宁波}']

In [164]:
model.predict(
    "宁波在哪个省" + " * " + "{'所在省': '所在省'}" + "|" + "{'宁波': '宁波'}"
)

['select distinct?answer where {?answer wdt:所在_省 wd:宁波}']

In [165]:
model.predict(
    "碳酸钙的副产品是什么? 在温度为1000的时候? * {'by-product': '副产物'}|{'temperature': '室温'}|{'calcium carbonate': '碳酸钙'}"
)

["SELECT?value WHERE { wd:calcium_carbonate p:temperature?s.?s ps:temperature?x filter(contains(?x,'1.000')).?s pq:by-product?value}"]

In [166]:
model.predict(
    "碳酸钙的副产品是什么? 在温度为1000的时候? * {'副产物': '副产物'}|{'室温': '室温'}|{'碳酸钙': '碳酸钙'}"
)

['SELECT?value WHERE { wd:碳酸钙 p:副产物?s.?s ps:副产物 wd:室温.?s pq:室温?value}']

In [169]:
model.predict(
    "哪个省管辖丰县？"
)

['select distinct?answer where {?answer wdt:哪个_省管辖льцi wd:Curtis}']

In [168]:
model.predict(
    "碳酸钙的副产品在温度为1000的时候是什么? * {'副产物': '副产物'}|{'室温': '室温'}|{'碳酸钙': '碳酸钙'}"
)

['SELECT?value WHERE { wd:碳酸钙 p:副产物?s.?s ps:副产物 wd:室温.?s pq:室温?value}']

In [167]:
model.predict(
    "碳酸钙的副产品是什么? 在温度为1000的时候? * {'碳酸钙': '碳酸钙'}|{'副产物': '副产物'}|{'室温': '室温'}"
)

['SELECT?value WHERE { wd:碳酸钙 p:副产物?s.?s ps:副产物 wd:副产物.?s pq:室温?value}']

In [154]:
model.predict(
    "宁波在哪个省" + " * " + "{'Ningbo': '宁波'}" + "|" + "{'in': '所在省'}" + "|" + "{'département': '省'}"
)

['SELECT?obj WHERE { wd:Ningbo p:département?s.?s ps:département?obj.?s pq:in wd:in }']

In [155]:
model.predict(
    "宁波在哪个省"
)

['select distinct?answer where { wd:宁波 wdt:鄞_在住_in_哪个_省u_.?answer}']

In [156]:
model.predict(
    "碳酸钙的副产品是什么? 在温度为1000的时候?"
)

['select distinct?answer where { wd:碳酸钙 wdt:temperature_accepted?answer}']

In [157]:
model.predict(
    "哪个职业是乔治布莱尔的人生巅峰?"
)

['select distinct?answer where { wd:John_Bush wdt:matches_of?answer}']

In [158]:
model.predict(
    "碳酸钙的副产品是什么?"
)

['select distinct?answer where { wd:calcium_carbonates wdt:副产品_is_?answer}']

In [ ]:
'select distinct?answer where {wd:Ningbo wdt:in ?answer}'

In [ ]:
model.predict(
    "宁波在哪个省" + " * " + "{'Ningbo': '宁波'}" + "|" + "{'in': '所在省'}" + "|" + 
)

In [ ]:
model.predict(
    "宁波在哪个省" + " * " + "{'Ningbo': '宁波'}" + "|" + "{'in': '所在省'}"
)

In [115]:
property_info_df[
    property_info_df["pid"] == "P131"
]

,pid,en_info,zh_info
4,P131,located in the administrative territorial enti...,所在行政領土實體 所在行政领土实体 隶属 上级行政区 行政區 隸屬行政區 上級行政區 所屬行...


In [88]:
iter_sent_on_db(sent)

{'碳酸': ['carbonic acid',
  'Acid of air',
  'Aerial acid',
  'Bisodium carbonate',
  'Calcined',
  'Carbonic acid sodium salt',
  'Consal',
  'Crystol carbonate',
  'Disodium carbonate',
  'H2CO3',
  'Mild alkali',
  'Na-X',
  'Oxyper',
  'Sal soda',
  'Salt of soda',
  'Scotch soda',
  'Soda ash',
  'Soda',
  'Sodium carbonate anhydrous',
  'Sodium carbonate hydrated',
  'Sodium carbonate peroxyhydrate',
  'Sodium carbonate',
  'Solvay soda',
  'Trona soda ash',
  'Tronalight light soda ash',
  '[CO(OH)2]'],
 '产品': ['product', 'goods', 'merchandise'],
 '温度': ['temperature']}

In [89]:
iter_sent_on_db("宁波在哪个省？")

{'宁波': ['Ningbo', 'Ningpo'], '省': ['department of France', 'département']}

In [90]:
iter_sent_on_db("美国的货币是什么？")

{'货币': [],
 '美国': ['United States of America',
  'America',
  'U.S.',
  'U.S.A.',
  'US',
  'USA',
  'United States',
  'the States',
  'the US',
  '🇺🇸']}

In [91]:
iter_sent_on_db("川普什么时候出生？")

{'川普': ['Donald Trump',
  'Donald J. Trump',
  'Donald John Trump',
  'The Donald',
  'Trump'],
 '出生': ['parturition', 'Birth', 'GO:0007567', 'birth', 'giving birth']}

In [49]:
####

In [65]:
zh_str = '碳酸'
g = wiki_entity_db.query("select s from en_zh_so_search where o like '%{}%'".format(zh_str))
l = list(g)

In [66]:
len(l)

107

In [67]:
pd.Series(l)

0      {'s': 'Q1025288'}
1       {'s': 'Q104334'}
2      {'s': 'Q1047326'}
3      {'s': 'Q1073138'}
4       {'s': 'Q130336'}
             ...        
102    {'s': 'Q8343712'}
103    {'s': 'Q8343713'}
104    {'s': 'Q8343736'}
105    {'s': 'Q8343738'}
106    {'s': 'Q8773993'}
Length: 107, dtype: object

In [18]:
zh_str = '碳酸'
g = wiki_entity_db.query("select * from en_zh_so_search where o == '{}'".format(zh_str))
l = list(g)

In [19]:
len(l)

1

In [159]:
model.predict(
    "哪个职业是乔治布莱尔的人生巅峰? * {'occupation': '职业'}|{'highlight moment': '高光时刻'}｜{'George Black': '乔治布莱克'}"
)

['SELECT?obj WHERE { wd:George_Black p:occupation?s.?s ps:occupation?obj.?s pq:highlight_moment wd:occupation }']

In [160]:
model.predict(
    "哪个职业是乔治布莱尔的人生巅峰? * {'status': '身份'}|{'moment': '时刻'}｜{'Black': '布莱克'}"
)

['select distinct?obj where { wd:Black wdt:moment?obj.?obj wdt:not_a_type_but_is_instance wd:status }']

In [161]:
model.predict(
    "哪个职业是乔治布莱尔的人生巅峰? * {'身份': '身份'}|{'时刻': '时刻'}｜{'布莱克': '布莱克'}"
)

['select distinct?obj where { wd:布莱克 wdt:时刻?obj.?obj wdt:not_a_type_but_is_instance wd:身份 }']

In [ ]:
model.predict(
    "哪个职业是乔治布莱尔的人生巅峰? * {'身份': '身份'}|{'时刻': '时刻'}｜{'布莱克': '布莱克'}"
)